In [1]:
%cd ../..

c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1


In [2]:
import os
import shutil

import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

from pathlib import Path

from src.forecasting.ml_forecasting import (
    MissingValueConfig,
    calculate_metrics,
)
from src.utils import plotting_utils
from tqdm.autonotebook import tqdm
from src.forecasting.ml_forecasting import calculate_metrics
from src.utils import ts_utils
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

In [3]:
os.makedirs("imgs/chapter_14", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

## Utility Functions

In [4]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        )
    )
    return fig

In [5]:
from itertools import cycle


def plot_forecast(pred_df, forecast_columns, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [
        "rgba(" + ",".join([str(c) for c in plotting_utils.hex_to_rgb(c)]) + ",<alpha>)"
        for c in px.colors.qualitative.Plotly
    ]
    act_color = colors[0]
    colors = cycle(colors[1:])
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].energy_consumption,
            mode="lines",
            line=dict(color=act_color.replace("<alpha>", "0.9")),
            name="Actual Consumption",
        )
    )
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines",
                line=dict(dash="dot", color=next(colors).replace("<alpha>", "1")),
                name=display_col,
            )
        )
    return fig

def highlight_abs_min(s, props=''):
    return np.where(s == np.nanmin(np.abs(s.values)), props, '')

## Reading the data

In [6]:
try:
    #Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    # Read in the Validation dataset as test_df so that we predict on it
    test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    # test_df = pd.read_parquet(preprocessed/"block_0-7_test_missing_imputed_feature_engg.parquet")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

In [7]:
target = "energy_consumption"
index_cols = ["LCLid", "timestamp"]

In [8]:
# Setting the indices
train_df.set_index(index_cols, inplace=True, drop=False)
test_df.set_index(index_cols, inplace=True, drop=False)

# Running Transformer Models on a Sample Household

## Selecting the sample data and metrics

In [9]:
sample_train_df = train_df.xs("MAC000193")
sample_test_df = test_df.xs("MAC000193")
# Creating a pred_df with actuals
pred_df = pd.concat([sample_train_df[[target]], sample_test_df[[target]]])

Split Train into Train and Validation and combine everything together into a single dataframe

In [10]:
sample_val_df = sample_train_df.loc["2013-12"]
sample_train_df = sample_train_df.loc[:"2013-11"]

sample_train_df['type'] = "train"
sample_val_df['type'] = "val"
sample_test_df['type'] = "test"
sample_df = pd.concat([sample_train_df[[target, "type"]], sample_val_df[[target, "type"]], sample_test_df[[target, "type"]]])
sample_df.head()

,energy_consumption,type
timestamp,,
2012-01-01 00:00:00,0.368,train
2012-01-01 00:30:00,0.386,train
2012-01-01 01:00:00,0.170,train
2012-01-01 01:30:00,0.021,train
2012-01-01 02:00:00,0.038,train


### Loading the Forecast and metrics from Seq-2Seq RNN

In [11]:
try:
    pred_df = pd.read_pickle(output/"dl_seq_2_seq_w_attn_prediction_val_df_MAC000193.pkl")
    metric_record = joblib.load(output/"dl_seq_2_seq_w_attn_metrics_val_df_MAC000193.pkl")

    sel_metrics = [
        "MultiStep LSTM_LSTM_teacher_forcing_1",
        "MultiStep_Seq2Seq_dot_Attn_teacher_forcing_1",
    ]
    metric_record = [i for i in metric_record if i["Algorithm"] in sel_metrics]
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Seq2Seq RNN with Attention.ipynb in Chapter14
    </div>
    """))

## Loading the necessary classes

In [12]:
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.models import TransformerConfig, TransformerModel
import pytorch_lightning as pl
import torch
# For reproduceability set a random seed
pl.seed_everything(42)

Seed set to 42


42

## Multi-Step Prediction

In [13]:
HORIZON = 48
WINDOW = 48*2

In [14]:
datamodule = TimeSeriesDataModule(data = sample_df[[target]],
        n_val = sample_val_df.shape[0],
        n_test = sample_test_df.shape[0],
        window = WINDOW, 
        horizon = HORIZON,
        normalize = "global", # normalizing the data
        batch_size = 32,
        num_workers = 0)
datamodule.setup()

### Transformer Model with Multi-step FF Decoder

In [15]:
transformer_config = TransformerConfig(
    input_size=1,
    d_model=64,
    n_heads=4,
    n_layers=2,
    ff_multiplier=4,
    dropout=0.1,
    activation="relu",
    multi_step_horizon=HORIZON,
    learning_rate=1e-3,
)

model = TransformerModel(config=transformer_config)

trainer = pl.Trainer(
    accelerator="auto",
    min_epochs=5,
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor="valid_loss", patience=3)],
)
trainer.fit(model, datamodule)
# model.load_state_dict(torch.load(trainer.checkpoint_callback.best_model_path)['state_dict'])
# Removing artifacts created during training
shutil.rmtree("lightning_logs")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1\lightning_logs

  | Name                | Type                    | Params 
-----------------------------------------------------------------
0 | input_projection    | Linear                  | 64     
1 | pos_encoder         | PositionalEncoding      | 0      
2 | encoder_layer       | TransformerEncoderLayer | 50.0 K 
3 | transformer_encoder | TransformerEncoder      | 100.0 K
4 | decoder             | Sequential              | 11.3 K 
5 | loss                | MSELoss                 | 0      
-----------------------------------------------------------------
161 K     Trainable params
0         Non-trainable params
161 K     Total params
0.645     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [16]:
tag = f"MultiStep_Transformer_Multi_Step_FF_decoder"
pred = trainer.predict(model, datamodule.test_dataloader())
# pred is a list of outputs, one for each batch
pred = torch.cat(pred).squeeze().detach().numpy()
# Selecting forward predictions of HORIZON timesteps, every HORIZON timesteps and flattening it
pred = pred[0::48].ravel()
# Apply reverse transformation because we applied global normalization
pred = pred * datamodule.train.std + datamodule.train.mean
pred_df_ = pd.DataFrame({tag: pred}, index=sample_test_df.index)
pred_df = pred_df.join(pred_df_)
metrics = calculate_metrics(sample_test_df[target], pred_df_[tag], tag, pd.concat([sample_train_df[target],sample_val_df[target]]))
# metrics
metric_record.append(metrics)

c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [17]:
formatted = pd.DataFrame(metric_record).style.format({"MAE": "{:.4f}", 
                          "MSE": "{:.4f}", 
                          "MASE": "{:.4f}", 
                          "Forecast Bias": "{:.2f}%"})
formatted.highlight_min(color='lightgreen', subset=["MAE","MSE","MASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,MASE,Forecast Bias
0,MultiStep LSTM_LSTM_teacher_forcing_1,0.1890,0.0985,1.4730,5.72%
1,MultiStep_Seq2Seq_dot_Attn_teacher_forcing_1,0.1580,0.0747,1.2310,5.39%
2,MultiStep_Transformer_Multi_Step_FF_decoder,0.2024,0.1096,1.5770,5.48%


In [18]:
fig = plot_forecast(pred_df, forecast_columns=[tag], forecast_display_names=[tag])
fig = format_plot(fig, title=f"MAE: {metrics['MAE']:.4f} | MSE: {metrics['MSE']:.4f} | MASE: {metrics['MASE']:.4f} | Bias: {metrics['Forecast Bias']:.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.write_image(f"imgs/chapter_13/{tag}.png")
fig.show()

In [19]:
# Removing artifacts created during training
shutil.rmtree("lightning_logs")

In [20]:
pred_df.to_pickle(output/"dl_transformers_val_df_MAC000193.pkl")
joblib.dump(metric_record, output/"dl_transformers_metrics_val_df_MAC000193.pkl")

['data\\london_smart_meters\\output\\dl_transformers_metrics_val_df_MAC000193.pkl']